In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

In [32]:
# Eli's code on median income + mine to get comparable Primary_MSA

median_income_csv = Path("../Data/household_income_msa_2019.csv")
median_income_df = pd.read_csv(median_income_csv)
median_income_df = median_income_df.transpose() # Transpose the column/headers
new_header = median_income_df.iloc[0] #grab the first row for the header
median_income_df = median_income_df[1:] #take the data less the header row
median_income_df.columns = new_header #set the header row as the df header
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] # Use only median and mean income data columns
median_income_df.reset_index(inplace=True)
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) # Split MSA from text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) # Split State from remaining text string
median_income_df[['Primary MSA1','Secondary MSA']] = median_income_df['MSA'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df[['Primary State','Secondary State']] = median_income_df['State'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df.drop(columns=['index','extra','extra2','MSA','State'],inplace=True) # Drop unnecessary columns
median_income_df.drop_duplicates(subset=['Primary MSA1','Primary State'], inplace=True) # Drop duplicates -- first instance is the median and mean income by MSA
median_income_df.head()

median_income_df[['Primary MSA']]= median_income_df['Primary MSA1']+"_"+median_income_df['Primary State']
median_income_df = median_income_df[["Primary MSA"]]
median_income_df.head()

Label,Primary MSA
0,Aberdeen_WA
8,Abilene_TX
16,Adrian_MI
24,Aguadilla_PR
32,Akron_OH


In [33]:
#checking NY MSA to compare with other datasets

checkmediandf = median_income_df.loc[median_income_df['Primary MSA'].str.contains('ND')]
checkmediandf

# has 18 rows of NY state MSA, weirdly enough does not include NYC

Label,Primary MSA
384,Bismarck_ND
1144,Fargo_ND
1360,Grand Forks_ND
2416,Minot_ND


In [40]:
# checking rental data from zillow

csv_path_2 = Path("../Data/rental_prices_msa.csv")
rental_price_2019 = pd.read_csv(csv_path_2)

#splitting City and State
rental_price_2019[['City','State']] = rental_price_2019.RegionName.str.split(",",expand=True)
rental_price_2019[['State']] = rental_price_2019['State'].str.strip()
rental_price_2019[['Primary MSA1','Secondary MSA']] = rental_price_2019['City'].str.split("-",n=1,expand=True)
rental_price_2019[['Primary MSA']]= rental_price_2019['Primary MSA1']+"_"+rental_price_2019['State']
rental_price_2019.drop(["RegionName","City","State","Primary MSA1", "Secondary MSA"], axis = 1, inplace = True)
rental_price_2019['Annual_Rent']= rental_price_2019.iloc[:, :-1].sum(axis=1)

rental_price_2019 = rental_price_2019[["Primary MSA"]]
rental_price_2019.dropna(subset = ["Primary MSA"], inplace=True)

checkrentaldf=rental_price_2019[rental_price_2019["Primary MSA"].str.contains('SD')]
checkrentaldf



,Primary MSA


In [41]:
# checking uscities data for long and lat

longlatdensity_csv = Path("../Data/uscities.csv")
longlatdensity_df = pd.read_csv(longlatdensity_csv)

pd.set_option('display.max_rows', None)


checkuscitiesdf=longlatdensity_df[longlatdensity_df['city'].str.contains('Fargo')]
checkuscitiesdf

#checkuscitiesdf = longlatdensity_df.loc[(longlatdensity_df.state_id == "NY")]
#checkuscitiesdf.head()

#This one has 1186 of long latitude for New York

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
215,Fargo,Fargo,ND,North Dakota,38017,Cass,46.8652,-96.8292,208675.0,967.0,polygon,False,True,America/Chicago,2,58102 58104 58103 58105 58106 58107 58108 5810...,1840000177
1324,West Fargo,West Fargo,ND,North Dakota,38017,Cass,46.8573,-96.9057,37058.0,892.0,polygon,False,True,America/Chicago,3,58078,1840001935
21795,Fargo,Fargo,OK,Oklahoma,40045,Ellis,36.3741,-99.6219,344.0,226.0,polygon,False,True,America/Chicago,3,73840,1840021653
22243,Fargo,Fargo,GA,Georgia,13065,Clinch,30.6881,-82.5721,316.0,44.0,polygon,False,True,America/New_York,3,31631,1840013893
27646,Fargo,Fargo,AR,Arkansas,5095,Monroe,34.9537,-91.1764,80.0,67.0,polygon,False,True,America/Chicago,3,72021,1840016506


In [42]:
# USING DATACOMPY LIBRARY TO COMPARE MEDIAN INC DATA AND ZILLOW RENTAL DATA

import datacompy
compare = datacompy.Compare(
rental_price_2019,
median_income_df,
join_columns='Primary MSA',  #You can also specify a list of columns eg ['policyID','statecode']
abs_tol=0, #Optional, defaults to 0
rel_tol=0, #Optional, defaults to 0
df1_name='Rental_DF', #Optional, defaults to 'df1'
df2_name='Median Income DF' #Optional, defaults to 'df2'
)
print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

          DataFrame  Columns  Rows
0         Rental_DF        1   105
1  Median Income DF        1   518

Column Summary
--------------

Number of columns in common: 1
Number of columns in Rental_DF but not in Median Income DF: 0
Number of columns in Median Income DF but not in Rental_DF: 0

Row Summary
-----------

Matched on: primary msa
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 99
Number of rows in Rental_DF but not in Median Income DF: 6
Number of rows in Median Income DF but not in Rental_DF: 419

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 99

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0

Sample Rows Only in Rental_DF (First 10 Columns)
--